In [20]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
import scipy.stats as st
import os.path
from os import path
import json

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_players = "output_data/players.csv"

# Range of latitudes and longitudes
# lat_range = (-90, 90)
# lng_range = (-180, 180)

In [2]:
def get_titledplayers_api(playertypes):
    url = "https://api.chess.com/pub/titled/"
    
    i = 0        
    players_data = []
    
    for playertype in playertypes:
        print(f"Index {str(i)} fetching player list for titled player type {playertype}")
    
        # Build query URL
        query_url = f"{url}{playertype}"

        # increment index counter
        i += 1

        # Get weather data    
        playertype_response = requests.get(query_url)
        playertype_json = playertype_response.json()
        players_data.append(playertype_json)        
        time.sleep(1)    
        
    return players_data

In [8]:
def get_playerdata_api(player_usernames):
    url = "https://api.chess.com/pub/player/"
    
    i = 0        
    player_data = []
    
    for player_username in player_usernames:
        print(f"Index {str(i)} fetching player data for player {player_username}")
    
        # Build query URL
        query_url = f"{url}{player_username}"

        # increment index counter
        i += 1

        # Get player data    
        try:
            player_response = requests.get(query_url)
            player_json = player_response.json()
            player_data.append(player_json)        
            time.sleep(1)    
        except:
            print("unexpected error")
        
        # if i > 10:
        #    break
    return player_data

In [9]:
titled_players_type = ["GM","WGM","IM","WIM","FM","WFM","NM","WNM","CM","WCM"]

In [10]:
playerslist_data = get_titledplayers_api(titled_players_type)

Index 0 fetching player list for titled player type GM
Index 1 fetching player list for titled player type WGM
Index 2 fetching player list for titled player type IM
Index 3 fetching player list for titled player type WIM
Index 4 fetching player list for titled player type FM
Index 5 fetching player list for titled player type WFM
Index 6 fetching player list for titled player type NM
Index 7 fetching player list for titled player type WNM
Index 8 fetching player list for titled player type CM
Index 9 fetching player list for titled player type WCM


In [11]:
playerlist_index = 0
i = 0

player_usernames = []

for playerlist in playerslist_data:    
    for player in playerlist["players"]:        
        player_usernames.append(player)
        
    playerlist_index += 1
    

```{
  "avatar": "https://images.chesscomfiles.com/uploads/v1/user/14775486.9072453d.200x200o.50bb2d10a7d5.jpeg",
  "player_id": 14775486,
  "@id": "https://api.chess.com/pub/player/140mariam",
  "url": "https://www.chess.com/member/140mariam",
  "name": "Mariam Avetisyan",
  "username": "140mariam",
  "title": "WFM",
  "followers": 56,
  "country": "https://api.chess.com/pub/country/AM",
  "last_online": 1609780423,
  "joined": 1385829847,
  "status": "premium",
  "is_streamer": false
}
```

In [12]:
len(player_usernames)

9302

In [13]:
playerdata = get_playerdata_api(player_usernames)

Index 0 fetching player data for player 123lt
Index 1 fetching player data for player 124chess
Index 2 fetching player data for player 1977ivan
Index 3 fetching player data for player 1stsecond
Index 4 fetching player data for player 2nd_life
Index 5 fetching player data for player 4thd-alpeacefulmoon
Index 6 fetching player data for player 64aramis64
Index 7 fetching player data for player 64arthos64
Index 8 fetching player data for player 64atilla64
Index 9 fetching player data for player 64dartagnan64
Index 10 fetching player data for player 64genghis64
Index 11 fetching player data for player 64leonidas64
Index 12 fetching player data for player 64porthos64
Index 13 fetching player data for player 64pyrrhus64
Index 14 fetching player data for player 731291
Index 15 fetching player data for player a-adly
Index 16 fetching player data for player a-fier
Index 17 fetching player data for player abasovn
Index 18 fetching player data for player abhidabhi
Index 19 fetching player data for

Index 156 fetching player data for player bfinegold-stl
Index 157 fetching player data for player bhat-sf
Index 158 fetching player data for player bigfish1995
Index 159 fetching player data for player bilelou
Index 160 fetching player data for player bilguunsu
Index 161 fetching player data for player billiekimbah
Index 162 fetching player data for player bilodeaua
Index 163 fetching player data for player blagid
Index 164 fetching player data for player blindfoldkrikor
Index 165 fetching player data for player blindginger
Index 166 fetching player data for player blindlast7samurai
Index 167 fetching player data for player blindtakes
Index 168 fetching player data for player bluewizzard
Index 169 fetching player data for player bobanbogosavljevic
Index 170 fetching player data for player bobbyhall222
Index 171 fetching player data for player bocah_sakit
Index 172 fetching player data for player bocharovd
Index 173 fetching player data for player bogdandeac
Index 174 fetching player da

Index 310 fetching player data for player dostis
Index 311 fetching player data for player dr-bassem
Index 312 fetching player data for player dr-cro
Index 313 fetching player data for player dragansolak
Index 314 fetching player data for player dragonflow
Index 315 fetching player data for player drags95
Index 316 fetching player data for player drblinditsky
Index 317 fetching player data for player drenchev
Index 318 fetching player data for player dretch
Index 319 fetching player data for player dropstonedp
Index 320 fetching player data for player drtancredi
Index 321 fetching player data for player drvelja
Index 322 fetching player data for player drvitman
Index 323 fetching player data for player drycounty
Index 324 fetching player data for player dr_mortimer
Index 325 fetching player data for player dudedoel
Index 326 fetching player data for player duhless
Index 327 fetching player data for player dulemaster
Index 328 fetching player data for player dulemudule
Index 329 fetchin

Index 464 fetching player data for player gmbenjaminfinegold
Index 465 fetching player data for player gmcheparinov
Index 466 fetching player data for player gmchessrob
Index 467 fetching player data for player gmcorrales
Index 468 fetching player data for player gmcristhian
Index 469 fetching player data for player gmcyborek
Index 470 fetching player data for player gmdavenogood
Index 471 fetching player data for player gmdrh
Index 472 fetching player data for player gmedmundo
Index 473 fetching player data for player gmg
Index 474 fetching player data for player gmgamil
Index 475 fetching player data for player gmharikrishna
Index 476 fetching player data for player gmhellers
Index 477 fetching player data for player gmhess
Index 478 fetching player data for player gmhikaruontwitch
Index 479 fetching player data for player gmianrogers
Index 480 fetching player data for player gmizeta
Index 481 fetching player data for player gmjacobaagaard
Index 482 fetching player data for player gm

Index 617 fetching player data for player igorkovalenko
Index 618 fetching player data for player igor_lysyj
Index 619 fetching player data for player igrok3069
Index 620 fetching player data for player ikar20
Index 621 fetching player data for player iljazaragatski
Index 622 fetching player data for player iljushin_alexey
Index 623 fetching player data for player illingworth
Index 624 fetching player data for player immatt64
Index 625 fetching player data for player imre91
Index 626 fetching player data for player incognito_knight
Index 627 fetching player data for player incrediblebubacik
Index 628 fetching player data for player indian-elephant
Index 629 fetching player data for player indianelephant5
Index 630 fetching player data for player indianlad
Index 631 fetching player data for player infernal_xam
Index 632 fetching player data for player injazzstyle
Index 633 fetching player data for player inopov
Index 634 fetching player data for player inspektor_rex
Index 635 fetching p

...

Index 8651 fetching player data for player kinga2008
Index 8652 fetching player data for player kingbwil
Index 8653 fetching player data for player kingcyber19
Index 8654 fetching player data for player kingrazor33
Index 8655 fetching player data for player kingscrusher
Index 8656 fetching player data for player king_kinto
Index 8657 fetching player data for player king_of_the_wing
Index 8658 fetching player data for player kirillgenius
Index 8659 fetching player data for player kirillov-dmitry
Index 8660 fetching player data for player kjellpetersson
Index 8661 fetching player data for player klaas1996
Index 8662 fetching player data for player kmpk
Index 8663 fetching player data for player kncp_719
Index 8664 fetching player data for player kngopal
Index 8665 fetching player data for player knightoutpostop
Index 8666 fetching player data for player kobemagic
Index 8667 fetching player data for player koneed
Index 8668 fetching player data for player korkmaz16
Index 8669 fetching pla

Index 8804 fetching player data for player patagonico2020
Index 8805 fetching player data for player patov
Index 8806 fetching player data for player patrisiii123
Index 8807 fetching player data for player pavelalexandru07
Index 8808 fetching player data for player pchelkinvk
Index 8809 fetching player data for player pedrosuarez
Index 8810 fetching player data for player pendragon619
Index 8811 fetching player data for player perecke
Index 8812 fetching player data for player perligero
Index 8813 fetching player data for player pertinhez76
Index 8814 fetching player data for player pesaruba
Index 8815 fetching player data for player peterflermoen
Index 8816 fetching player data for player peternsw
Index 8817 fetching player data for player pgrj
Index 8818 fetching player data for player philekhoob
Index 8819 fetching player data for player phriex
Index 8820 fetching player data for player phurba13
Index 8821 fetching player data for player piccadilly123
Index 8822 fetching player data

Index 8954 fetching player data for player suleymanontwitch
Index 8955 fetching player data for player suleyman_chess06
Index 8956 fetching player data for player sultaichyngyz
Index 8957 fetching player data for player summertimefun
Index 8958 fetching player data for player superboemer
Index 8959 fetching player data for player supermagnum
Index 8960 fetching player data for player supzuroke
Index 8961 fetching player data for player svet2008
Index 8962 fetching player data for player tabarex
Index 8963 fetching player data for player taildeitali
Index 8964 fetching player data for player talyamania
Index 8965 fetching player data for player taran2004
Index 8966 fetching player data for player tastyfrenchfries
Index 8967 fetching player data for player tavi001
Index 8968 fetching player data for player teamempire
Index 8969 fetching player data for player tengischess
Index 8970 fetching player data for player tenorespinto
Index 8971 fetching player data for player tepcovua2007
Index 

Index 9106 fetching player data for player conmbatha
Index 9107 fetching player data for player czernikowskqa
Index 9108 fetching player data for player daia93
Index 9109 fetching player data for player deepfear
Index 9110 fetching player data for player delfin000
Index 9111 fetching player data for player delfina20jo
Index 9112 fetching player data for player denguvi
Index 9113 fetching player data for player destroyermassive
Index 9114 fetching player data for player detectiveorange
Index 9115 fetching player data for player dianalomaia1
Index 9116 fetching player data for player digilik
Index 9117 fetching player data for player econ210
Index 9118 fetching player data for player emka9335
Index 9119 fetching player data for player enen0315
Index 9120 fetching player data for player ernestina08chess
Index 9121 fetching player data for player etcoco
Index 9122 fetching player data for player evachess2006
Index 9123 fetching player data for player evitacher
Index 9124 fetching player da

Index 9257 fetching player data for player savvyshark
Index 9258 fetching player data for player shellymilly
Index 9259 fetching player data for player shemilah80
Index 9260 fetching player data for player sherry_ali
Index 9261 fetching player data for player shiningdiamond
Index 9262 fetching player data for player shriyanasmallya
Index 9263 fetching player data for player silvercat1992
Index 9264 fetching player data for player sofiia_hryzlova
Index 9265 fetching player data for player solar_ao
Index 9266 fetching player data for player spacegirl987
Index 9267 fetching player data for player speshka
Index 9268 fetching player data for player ssunu
Index 9269 fetching player data for player strukovaksenia
Index 9270 fetching player data for player sukhugloria
Index 9271 fetching player data for player summer-storm
Index 9272 fetching player data for player sunshine050
Index 9273 fetching player data for player tae_gyeong_kim
Index 9274 fetching player data for player thornbird2020
Ind

In [ ]:
len(playerdata[0])
len(playerdata[1])

In [14]:
player_id = []
username = []
title = []
followers = []
country = []
location = []
last_online = []
joined = []
status = []
is_streamer = []

i = 0

for player in playerdata: 
    try:
        un = player["username"]
        print(f"Index {i} processing player {un}")
        player_id.append(player["player_id"])
        username.append(player["username"])
        title.append(player["title"])
        followers.append(player["followers"])
        country.append(player["country"].replace("https://api.chess.com/pub/country/","")),
        if "location" in player.keys():
            location.append(player["location"])        
        else:
            location.append("")                
        last_online.append(player["last_online"])
        joined.append(player["joined"])
        status.append(player["status"])
        is_streamer.append(player["is_streamer"])
    except:
        print("unexpected error")
        
    i += 1    

Index 0 processing player 123lt
Index 1 processing player 124chess
Index 2 processing player 1977ivan
Index 3 processing player 1stsecond
Index 4 processing player 2nd_life
Index 5 processing player 4thd-alpeacefulmoon
Index 6 processing player 64aramis64
Index 7 processing player 64arthos64
Index 8 processing player 64atilla64
Index 9 processing player 64dartagnan64
Index 10 processing player 64genghis64
Index 11 processing player 64leonidas64
Index 12 processing player 64porthos64
Index 13 processing player 64pyrrhus64
Index 14 processing player 731291
Index 15 processing player a-adly
Index 16 processing player a-fier
Index 17 processing player abasovn
Index 18 processing player abhidabhi
Index 19 processing player abhijeetgupta
Index 20 processing player abhijeetgupta1016
Index 21 processing player abhijeetonyoutube
Index 22 processing player abhyak
Index 23 processing player abykhovsky
Index 24 processing player actorxu
Index 25 processing player adaro
Index 26 processing player a

Index 2922 processing player sestertii
Index 2923 processing player sestremera
Index 2924 processing player sever81
Index 2925 processing player sganerdene
Index 2926 processing player sgchess01
Index 2927 processing player shadowfighter64
Index 2928 processing player shahaliyev
Index 2929 processing player shahinyan_chessmood
Index 2930 processing player shambala777
Index 2931 processing player shankovskyi
Index 2932 processing player sharafiev_azat
Index 2933 processing player shav3184
Index 2934 processing player shayporat
Index 2935 processing player shensai
Index 2936 processing player shepherdboy
Index 2937 processing player sherlok7
Index 2938 processing player shilichkov
Index 2939 processing player shineatchess
Index 2940 processing player shinya_kojima
Index 2941 processing player shivenkhosla
Index 2942 processing player shoorly
Index 2943 processing player shosti-73
Index 2944 processing player shota_azaladze
Index 2945 processing player shoytan
Index 2946 processing player

Index 5921 processing player szapatacharles
Index 5922 processing player szeprozse
Index 5923 processing player szmetan
Index 5924 processing player szmm19
Index 5925 processing player s_kalvaitis
Index 5926 processing player tachion
Index 5927 processing player tacticthunder
Index 5928 processing player tactictic
Index 5929 processing player tacticus_severus
Index 5930 processing player tagliata
Index 5931 processing player takemypieces97
Index 5932 processing player taktikmonster
Index 5933 processing player taktychenko
Index 5934 processing player talaris
Index 5935 processing player talento10
Index 5936 processing player talman67
Index 5937 processing player tamamm
Index 5938 processing player tamas_vanczak
Index 5939 processing player tamben
Index 5940 processing player tampachess
Index 5941 processing player tandaleo
Index 5942 processing player tanilx
Index 5943 processing player tao7
Index 5944 processing player tarantul1
Index 5945 processing player taron_shagbazyan
Index 5946

In [15]:
player_dict = {
    "player_id":player_id,
    "username":username,
    "title":title,
    "followers":followers,
    "country":country,
    "location":location,
    "last_online":last_online,
    "joined":joined,
    "status":status,
    "is_streamer":is_streamer
}
chessplayer_df = pd.DataFrame(player_dict)


In [16]:
chessplayer_df

,player_id,username,title,followers,country,location,last_online,joined,status,is_streamer
0,18800602,123lt,GM,32,CN,,1608465021,1410059361,premium,False
1,29499974,124chess,GM,79,RU,Красноярск,1609836300,1471316272,premium,False
2,30610578,1977ivan,GM,95,RS,Belgrade,1609872690,1477565847,premium,False
3,13013662,1stsecond,GM,734,LV,Riga,1609536323,1375617889,premium,False
4,39230304,2nd_life,GM,22,CH,Zurich,1609876221,1507311109,premium,False
...,...,...,...,...,...,...,...,...,...,...
9295,29980036,yashami,WCM,21,NP,Kathmandu,1594232302,1474135436,premium,False
9296,67624176,yoriiallam,WCM,10,EG,,1609501122,1573057158,premium,False
9297,86574494,zaineb218,WCM,3,TN,Tunisie,1609446669,1594630390,premium,False
9298,85888234,zarinur,WCM,15,KZ,,1609904817,1594013985,premium,False


In [21]:
chessplayer_df.to_csv(output_data_players, index=True,index_label="player_index")